## Make the dataframe including the prices of agriculture products for the each years and regions
- Date : Mar 21, 2023
- Author : JaeEun Yoo

In [6]:
import os
import glob
import requests
import json
import pandas as pd
from urllib.request import urlopen
from itertools import repeat

## Read the code information to use the open API

In [7]:
file_path = '../샘플데이터/02. 농산물유통정보시스템_농작물 도소매가격(apen api)/농축수산물 품목 및 등급 코드표.xlsx'
# Get the sheet names of files
code_file = pd.read_excel(file_path,header=1,sheet_name = None)
sheet_names = [k for k in code_file.keys()]
sheet_names

['부류코드', '품목코드', '품종코드', '코드통합(부류＋품목＋품종코드)', '등급코드']

In [8]:
# Load the category code and grade code
cate_code = pd.read_excel(file_path,header=1,sheet_name = sheet_names[-2])
grade_code = pd.read_excel(file_path,header=1,sheet_name = sheet_names[-1])

In [9]:
# Get only dataset
cate_code = cate_code[cate_code.keys()[1:10]]
grade_code = grade_code[grade_code.keys()[1:-1]]
grade_code = grade_code[:-4]
grade_list = [str(str(int(x)).zfill(2)) for x in grade_code['등급코드(p_productrankcode)']]

In [10]:
cate_code

,품목 그룹코드,품목 그룹명,품목 코드,품목명,품종코드,품종명,도매출하단위,도매출하단위 크기,소매출하단위
0,100,식량작물,111,쌀,1,일반계,kg,20.0,kg
1,100,식량작물,111,쌀,2,백미,NaN,NaN,NaN
2,100,식량작물,111,쌀,3,현미,NaN,NaN,NaN
3,100,식량작물,111,쌀,5,햇일반계,kg,20.0,kg
4,100,식량작물,112,찹쌀,1,일반계,kg,40.0,kg
...,...,...,...,...,...,...,...,...,...
199,600,수산물,650,새우젓,0,새우젓,NaN,NaN,kg
200,600,수산물,651,멸치액젓,0,멸치액젓,NaN,NaN,kg
201,600,수산물,652,굵은소금,0,굵은소금,NaN,NaN,kg
202,600,수산물,653,전복,0,전복,kg,1.0,마리


## Initialize the parameters

In [24]:
DOC_TYPE = 'json'
API_URL = 'http://www.kamis.or.kr/service/price/xml.do?action=periodProductList'
START_DATE = '2022-03-21'
END_DATE='2023-03-20'
API_KEY = 'c79c1c37-7b6d-472c-83e4-a967c29d0bfe'

# Retails
retail = {'1101' : '서울특별시', 
          '2100' : '부산광역시', 
          '2200' : '대구광역시',
          '2300' : '인천광역시', 
          '2401' : '광주광역시', 
          '2501' : '대전광역시', 
          '2601' : '울산광역시', 
          '3111' : '경기도 수원시', 
          '3211' : '강원도 춘천시', 
          '3311' : '충청북도 청주시', 
          '3511' : '전라북도 전주시', 
          '3711' : '경상북도 포항시', 
          '3911' : '제주특별자치도', 
          '3113' : '서울특별시 의정부', 
          '3613' : '전라남도 순천시', 
          '3714' : '경상북도 안동시', 
          '3814' : '경상남도 창원시', 
          '3145' : '경기도 용인시'}

# Wholes
wholes ={'1101' : '서울특별시', 
          '2100' : '부산광역시', 
          '2200' : '대구광역시', 
          '2401' : '광주광역시', 
          '2501' : '대전광역시'}

# 소매 및 도매 여부
product_clscode = {'01':'소매','02':'도매'}

## Run

In [50]:
clscode = '02'
# Grade
for grade in grade_list:
    # Country
    for country in wholes.keys():
        # Codes
        for p_itemcategorycode,p_itemcode, p_kindcode in zip(cate_code['품목 그룹코드'],
                                                             cate_code['품목 코드'],
                                                             cate_code['품종코드']):

            # Define the parameters
            params ={'p_productclscode' : clscode, 
                     'p_startday' : START_DATE, 
                     'p_endday' : END_DATE,
                     'p_itemcategorycode' : str(p_itemcategorycode),
                     'p_itemcode' : str(p_itemcode), 
                     'p_kindcode' : str(str(int(p_kindcode)).zfill(2)),
                     'p_productrankcode' : grade,
                     'p_countrycode':country,
                     'p_convert_kg_yn' : 'Y',
                     'p_cert_key' : API_KEY,
                     'p_cert_id' : '3220',
                     'p_returntype' : DOC_TYPE
                    }

            # API request
            response = requests.get(API_URL, params=params)
            # Check the status code
            res = response.content.decode('utf-8')
            
            # If not error code is 500
            if len(res)!=0:
                # Change the string to json
                json_acceptable_string = res.replace("'", "\"")
                res = json.loads(json_acceptable_string)
                print(res)
                # If the json is OK
                if res['data'][0]!='001':
                    print(len(res))
                    break
                else:
                    pass



{'condition': [{'p_startday': '2022-03-23', 'p_endday': '2023-03-20', 'p_itemcategorycode': '100', 'p_itemcode': '111', 'p_kindcode': '01', 'p_productrankcode': '01', 'p_countycode': '1101', 'p_convert_kg_yn': 'Y', 'p_key': 'c79c1c37-7b6d-472c-83e4-a967c29d0bfe', 'p_id': '3220', 'p_returntype': 'json'}], 'data': {'error_code': '000', 'item': [{'itemname': [], 'kindname': [], 'countyname': '평균', 'marketname': [], 'yyyy': '2022', 'regday': '03/23', 'price': '2,365'}, {'itemname': [], 'kindname': [], 'countyname': '평균', 'marketname': [], 'yyyy': '2022', 'regday': '03/24', 'price': '2,365'}, {'itemname': [], 'kindname': [], 'countyname': '평균', 'marketname': [], 'yyyy': '2022', 'regday': '03/25', 'price': '2,365'}, {'itemname': [], 'kindname': [], 'countyname': '평균', 'marketname': [], 'yyyy': '2022', 'regday': '03/28', 'price': '2,365'}, {'itemname': [], 'kindname': [], 'countyname': '평균', 'marketname': [], 'yyyy': '2022', 'regday': '03/29', 'price': '2,365'}, {'itemname': [], 'kindname': [

KeyError: 0

In [51]:
# Split the dataset (average, common, item prices for the each days)
avg = res['data']['item'][:int(len(res['data']['item'])/3)]
com = res['data']['item'][int(len(res['data']['item'])/3):int(len(res['data']['item'])/3*2)]
items = res['data']['item'][int(len(res['data']['item'])/3*2):]

In [52]:
# Items dataframe
df = pd.DataFrame(items)
df['Date'] = df['yyyy'] +'/' +  df['regday']
df = df.reindex(columns = ['Date','itemname','kindname','countyname','marketname','price'])
df.columns = ['Date','품목명','품종명','지역','시장','가격(원)']
w_r = pd.DataFrame(data=list(repeat(product_clscode[clscode],len(df))),columns=['도/소매']).reset_index(drop=True)
df = pd.concat([df,w_r],axis=1)
        
    
# Common prices dataframe
tmp_com = pd.DataFrame(com)
tmp_com['Date'] = tmp_com['yyyy'] +'/' +  tmp_com['regday']
tmp_com = tmp_com.reindex(columns = ['Date','price'])
tmp_com.columns = ['Date','평년가격(원)']

# Avarage prices dataframe
tmp_avg = pd.DataFrame(avg)
tmp_avg['Date'] = tmp_avg['yyyy'] +'/' +  tmp_avg['regday']
tmp_avg = tmp_avg.reindex(columns = ['Date','price'])
tmp_avg.columns = ['Date','평균가격(원)']



In [53]:
results = pd.merge(df,tmp_com,how='left',on='Date')
results = pd.merge(results,tmp_avg,how='left',on='Date')

In [49]:
t = results.copy()

In [5]:
t

,Date,품목명,품종명,지역,시장,가격,도/소매,평년 가격,평균 가격
0,2022/03/21,콩,흰 콩(국산)(1kg),서울,경동,"6,600",소매,NaN,"6,600"
1,2022/03/22,콩,흰 콩(국산)(1kg),서울,경동,"6,600",소매,NaN,"6,600"
2,2022/03/23,콩,흰 콩(국산)(1kg),서울,경동,"6,600",소매,NaN,"6,600"
3,2022/03/24,콩,흰 콩(국산)(1kg),서울,경동,"6,600",소매,NaN,"6,600"
4,2022/03/25,콩,흰 콩(국산)(1kg),서울,경동,"6,600",소매,NaN,"6,600"
...,...,...,...,...,...,...,...,...,...
302,2023/03/14,쌀,일반계(1kg),서울,양곡도매,"2,050",도매,"2,347","2,050"
303,2023/03/15,쌀,일반계(1kg),서울,양곡도매,"2,050",도매,"2,346","2,050"
304,2023/03/16,쌀,일반계(1kg),서울,양곡도매,"2,050",도매,"2,344","2,050"
305,2023/03/17,쌀,일반계(1kg),서울,양곡도매,"2,050",도매,"2,342","2,050"


In [55]:
results = pd.concat([t,results])

In [56]:
results.to_csv('../results_data/02_농산물도소매가격_'+START_DATE+'_'+END_DATE+'.csv',index=False,encoding='cp949')